In [12]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json
import pandas as pd

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   

# Load data

In [2]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs_eq.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)


In [3]:
row_names = list(edges.keys())
n_state = 2 # binary states

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

In [4]:
dests = ['n22', 'n66']
sys_surv_st = 2

def s_fun(comps_st):
    conn_pop_ratio, sys_st, info = fbg.eval_population_accessibility(comps_st, G_base, dests,
                                                         avg_speed=60.0, # km/h
                                                         target_time_max = 0.25, # hours: it shouldn't take longer than this to reach any destination
                                                         target_pop_max = [0.80, 0.95], # fraction of population that should be reachable at each destination
                                                         length_attr = 'length_km',
                                                         population_attr = 'population',)

    min_comps_st = None
    return conn_pop_ratio, sys_st, min_comps_st

row_names = list(edges.keys()) 
n_state = 2 # binary states of components

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)


In [5]:
TSUMPATH = Path("tsum_res") 

rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_st}.pt", map_location="cpu")
rules_mat_surv = rules_mat_surv.to(device)
rules_mat_fail = torch.load(TSUMPATH / f"rules_leq_{sys_surv_st-1}.pt", map_location="cpu")
rules_mat_fail = rules_mat_fail.to(device)

# Calculate system probabilities

## Marginal probability

In [6]:
pr_cond = tsum.get_comp_cond_sys_prob(
    rules_mat_surv,
    rules_mat_fail,
    probs,
    comps_st_cond = {},
    row_names = row_names,
    s_fun = s_fun,
    sys_surv_st = sys_surv_st
)
print(f"P(sys >= {sys_surv_st}) = {pr_cond['survival']:.3e}")
print(f"P(sys <= {sys_surv_st-1} ) = {pr_cond['failure']:.3e}\n")

P(sys >= 2) = 8.012e-01
P(sys <= 1 ) = 1.988e-01



## Conditional probability given "one" component's survival

In [7]:
for x in row_names:
    print(f"Eval P(sys | {x}=1)")
    pr_cond = tsum.get_comp_cond_sys_prob(
        rules_mat_surv,
        rules_mat_fail,
        probs,
        comps_st_cond = {x: 1},
        row_names = row_names,
        s_fun = s_fun,
        sys_surv_st = sys_surv_st
    )
    print(f"P(sys >= {sys_surv_st} | {x}=1) = {pr_cond['survival']:.3e}")
    print(f"P(sys <= {sys_surv_st-1} | {x}=1) = {pr_cond['failure']:.3e}\n")

Eval P(sys | e0001=1)
P(sys >= 2 | e0001=1) = 8.012e-01
P(sys <= 1 | e0001=1) = 1.988e-01

Eval P(sys | e0002=1)
P(sys >= 2 | e0002=1) = 8.013e-01
P(sys <= 1 | e0002=1) = 1.987e-01

Eval P(sys | e0003=1)
P(sys >= 2 | e0003=1) = 8.015e-01
P(sys <= 1 | e0003=1) = 1.985e-01

Eval P(sys | e0004=1)
P(sys >= 2 | e0004=1) = 8.041e-01
P(sys <= 1 | e0004=1) = 1.959e-01

Eval P(sys | e0005=1)
P(sys >= 2 | e0005=1) = 8.014e-01
P(sys <= 1 | e0005=1) = 1.986e-01

Eval P(sys | e0006=1)
P(sys >= 2 | e0006=1) = 8.015e-01
P(sys <= 1 | e0006=1) = 1.985e-01

Eval P(sys | e0007=1)
P(sys >= 2 | e0007=1) = 8.020e-01
P(sys <= 1 | e0007=1) = 1.980e-01

Eval P(sys | e0008=1)
P(sys >= 2 | e0008=1) = 8.012e-01
P(sys <= 1 | e0008=1) = 1.988e-01

Eval P(sys | e0009=1)
P(sys >= 2 | e0009=1) = 8.014e-01
P(sys <= 1 | e0009=1) = 1.986e-01

Eval P(sys | e0010=1)
P(sys >= 2 | e0010=1) = 8.019e-01
P(sys <= 1 | e0010=1) = 1.981e-01

Eval P(sys | e0011=1)
P(sys >= 2 | e0011=1) = 8.018e-01
P(sys <= 1 | e0011=1) = 1.982e-01


# Get multi-state probabilities

## First load rules for all states

In [8]:
TSUMPATH = Path("tsum_res") 
rules_dict_mat_surv = {}
rules_dict_mat_fail = {}

for sys_surv_st in [1, 2]:  # either 1 or 2
    rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_st}.pt", map_location="cpu")
    rules_mat_surv = rules_mat_surv.to(device)
    rules_dict_mat_surv[sys_surv_st] = rules_mat_surv
    rules_mat_fail = torch.load(TSUMPATH / f"rules_leq_{sys_surv_st-1}.pt", map_location="cpu")
    rules_mat_fail = rules_mat_fail.to(device)
    rules_dict_mat_fail[sys_surv_st] = rules_mat_fail

## Marginal probability

In [ ]:
# Initialize empty list to store the results
results = []

# Calculate probabilities
cond_probs = tsum.get_comp_cond_sys_prob_multi(
                rules_dict_mat_surv,
                rules_dict_mat_fail,
                probs,
                comps_st_cond = {},
                row_names = row_names,
                s_fun = s_fun
            )

# Print results
print("P(sys):", cond_probs)

# Append data as a dictionary to the list
results.append({"System failure": cond_probs[0],
                "Partial failure": cond_probs[1],
                "Survival": cond_probs[2]
                })

# Convert the list to a DataFrame
df_results = pd.DataFrame(results)

# Save to a JSON file
df_results.to_json("data/sys_probs.json", orient="records", indent=4)

print("\nData saved to data/sys_probs.json")

P(sys): {0: 0.000448, 1: 0.197756, 2: 0.801796}

Data saved to post-processing/sys_probs.json


## Conditional probability given "one" component's survival

In [ ]:
# Initialize empty list to store the results
results = []

for x in row_names:
    # Calculate probabilities
    cond_probs = tsum.get_comp_cond_sys_prob_multi(
                    rules_dict_mat_surv,
                    rules_dict_mat_fail,
                    probs,
                    comps_st_cond = {x: 0}, # 1: survival, 0: failure
                    row_names = row_names,
                    s_fun = s_fun
                )

    # Print results
    print(f"P(sys | {x}=0):", cond_probs)

    # Append data as a dictionary to the list
    results.append({"Component": x,
                    "System failure": cond_probs[0],
                    "Partial failure": cond_probs[1],
                    "Survival": cond_probs[2]
                    })

# Convert the list to a DataFrame
df_results = pd.DataFrame(results)

# Save to a JSON file
df_results.to_json("data/cond_sys_probs.json", orient="records", indent=4)

print("\nData saved to data/cond_sys_probs.json")

P(sys | e0001=0): {0: 0.000415, 1: 0.196767, 2: 0.802818}
P(sys | e0002=0): {0: 0.000429, 1: 0.195971, 2: 0.8036}
P(sys | e0003=0): {0: 0.000431, 1: 0.197651, 2: 0.801918}
P(sys | e0004=0): {0: 0.00059, 1: 0.981887, 2: 0.017523}
P(sys | e0005=0): {0: 0.00395, 1: 0.192991, 2: 0.803059}
P(sys | e0006=0): {0: 0.000431, 1: 0.195508, 2: 0.804061}
P(sys | e0007=0): {0: 0.000426, 1: 0.197113, 2: 0.802461}
P(sys | e0008=0): {0: 0.00041, 1: 0.198601, 2: 0.800989}
P(sys | e0009=0): {0: 0.000399, 1: 0.197557, 2: 0.802044}
P(sys | e0010=0): {0: 0.000384, 1: 0.197919, 2: 0.801697}
P(sys | e0011=0): {0: 0.000455, 1: 0.198087, 2: 0.801458}
P(sys | e0012=0): {0: 0.000444, 1: 0.197589, 2: 0.801967}
P(sys | e0013=0): {0: 0.000412, 1: 0.196172, 2: 0.803416}
P(sys | e0014=0): {0: 0.000398, 1: 0.196989, 2: 0.802613}
P(sys | e0015=0): {0: 0.000416, 1: 0.197315, 2: 0.802269}
P(sys | e0016=0): {0: 0.000405, 1: 0.198081, 2: 0.801514}
P(sys | e0017=0): {0: 0.000412, 1: 0.196692, 2: 0.802896}
P(sys | e0018=0): {